# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Import CSV file and show data
weather_data_file = "../cities_weather.csv"
weather_data_df = pd.read_csv(weather_data_file)



weather_data_df


,Unnamed: 0,City Name,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Barrow,71.2906,-156.7887,30.22,74,90,9.22,US,1633801668
1,1,Atar,20.5169,-13.0499,99.61,10,99,11.45,MR,1633801670
2,2,Maceió,-9.6658,-35.7353,87.24,58,20,11.50,BR,1633801437
3,3,Gaya,24.7833,85.0000,78.71,94,5,0.00,IN,1633801672
4,4,Encantado,-29.2361,-51.8697,71.65,91,100,2.64,BR,1633801674
...,...,...,...,...,...,...,...,...,...,...
589,589,Pandan,14.0453,124.1694,80.55,89,99,16.98,PH,1633802579
590,590,Xai-Xai,-25.0519,33.6442,80.13,51,0,11.54,MZ,1633802580
591,591,Asaba,6.2006,6.7338,76.32,94,97,6.55,NG,1633802581
592,592,Kamenka,51.3223,42.7678,43.56,60,3,6.02,RU,1633802583


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#Store locations and humidity
locations = weather_data_df[["Lat", "Lon"]].astype(float)



humidity = weather_data_df["Humidity"].astype(float)





In [5]:
#Create heatmap with heat layer

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                  dissipating=False, max_intensity=100,
                                  point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
weather_data_df[weather_data_df.isna().any(axis=1)]

,Unnamed: 0,City Name,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
78,78,Oranjemund,-28.5500,16.4333,59.04,83,16,20.07,NaN,1633801795
116,116,Lüderitz,-26.6481,15.1594,66.18,56,14,22.35,NaN,1633801853
373,373,Aranos,-24.1333,19.1167,70.41,7,2,12.55,NaN,1633802245


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#Find ideal weather conditions and show data

ideal_city = weather_data_df.loc[(weather_data_df["Max Temp"] >= 70) & 
                                       (weather_data_df["Max Temp"] <=80) &
                                       (weather_data_df["Wind Speed"] < 10) &
                                       (weather_data_df["Cloudiness"] == 0)].dropna()

ideal_city
                                       
                               

,Unnamed: 0,City Name,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,Abu Samrah,35.3029,37.1841,76.42,22,0,3.47,SY,1633801686
74,74,Utraula,27.3167,82.4167,75.38,68,0,4.00,IN,1633801789
170,170,Tezu,27.9167,96.1667,75.87,78,0,3.38,IN,1633801937
264,264,Rawson,-43.3002,-65.1023,72.18,28,0,5.73,AR,1633801851
404,404,Nishifukuma,33.7667,130.4667,73.40,74,0,6.02,JP,1633802267
474,474,Kiryat Gat,31.6100,34.7642,76.82,65,0,8.59,IL,1633802343
482,482,Sonoita,31.8500,-112.8333,79.54,22,0,6.49,MX,1633802412
488,488,Galeana,24.8333,-100.0667,76.32,29,0,2.35,MX,1633802423
515,515,Beloha,-25.1667,45.0500,70.38,63,0,4.14,MG,1633802466
552,552,Kaita,34.3667,132.5500,71.17,85,0,1.61,JP,1633802521


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#Create data frame including new column for Hotel Name

ideal_city['Hotel Name'] = ""
hotel_df = ideal_city[["City Name", "Lat", "Lon","Humidity", "Country", "Hotel Name"]].copy()
hotel_df

,City Name,Lat,Lon,Humidity,Country,Hotel Name
12,Abu Samrah,35.3029,37.1841,22,SY,
74,Utraula,27.3167,82.4167,68,IN,
170,Tezu,27.9167,96.1667,78,IN,
264,Rawson,-43.3002,-65.1023,28,AR,
404,Nishifukuma,33.7667,130.4667,74,JP,
474,Kiryat Gat,31.6100,34.7642,65,IL,
482,Sonoita,31.8500,-112.8333,22,MX,
488,Galeana,24.8333,-100.0667,29,MX,
515,Beloha,-25.1667,45.0500,63,MG,
552,Kaita,34.3667,132.5500,85,JP,


In [9]:
#Loop through the dataframe and request first hotel name in the city

params = {"radius": 5000,
          "types": "lodging",
          "keyword": "hotel",
          "key": g_key}
city_name = []


for index, row in hotel_df.iterrows():
    
        
    lat = row["Lat"]
    lon = row["Lon"]
    
    params["location"] = f"{lat},{lon}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_name = requests.get(base_url, params=params)
    
    hotel_name = hotel_name.json()
    
    #print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    try:
    
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        #city_name.append(hotel_df["City Name"])
        print("No hotel found in ")
        pass
    
hotel_df
    
    
             
    




No hotel found in 
No hotel found in 
No hotel found in 
No hotel found in 


,City Name,Lat,Lon,Humidity,Country,Hotel Name
12,Abu Samrah,35.3029,37.1841,22,SY,
74,Utraula,27.3167,82.4167,68,IN,Bhallu Kallu Hotel
170,Tezu,27.9167,96.1667,78,IN,Oshin Hotel
264,Rawson,-43.3002,-65.1023,28,AR,Complejo Bahia Ballenas
404,Nishifukuma,33.7667,130.4667,74,JP,
474,Kiryat Gat,31.6100,34.7642,65,IL,Desert Gat
482,Sonoita,31.8500,-112.8333,22,MX,Hotel Excelsior
488,Galeana,24.8333,-100.0667,29,MX,Hotel Magdalena
515,Beloha,-25.1667,45.0500,63,MG,
552,Kaita,34.3667,132.5500,85,JP,


In [10]:
#Put hotel name into a list
hotels = hotel_df["Hotel Name"].tolist()

In [11]:
#Configure gmaps for ideal weather conditions dataframe
gmaps.configure(api_key=g_key)

In [12]:
#Store locations and humidity
locations = hotel_df[["Lat", "Lon"]].astype(float)


humidity = hotel_df["Humidity"].astype(float)

In [13]:
#Configure heat map with heat layer and hotel layer

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                  dissipating=False, max_intensity=100,
                                  point_radius = 1)



hotel_layer = gmaps.symbol_layer(locations, fill_color = 'rgba(0, 150, 0, 0.4)', stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
                                 info_box_content=[f"Hotel Name: {hotel}" for hotel in hotels])

fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)


fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# # NOTE: Do not change any of the code in this cell

# # Using the template add the hotel marks to the heatmap
# info_box_template = """
# <dl>
# <dt>Name</dt><dd>{Hotel Name}</dd>
# <dt>City</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# </dl>
# """
# # Store the DataFrame Row
# # NOTE: be sure to update with your DataFrame name
# hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# locations = hotel_df[["Lat", "Lng"]]